In [57]:
let ResolutionFolder = __SOURCE_DIRECTORY__
type Tree = {
    line: int
    column: int
    height: int
}

module Visible =
    type State = { max: int; visible: Tree seq } with static member Empty = { max = -1; visible = Seq.empty }
    let find (line:Tree seq) =
        let folder acc tree =
            if tree.height > acc.max then
                { max = tree.height; visible = acc.visible |> Seq.append [tree] }
            else
                acc
        let front = line |> Seq.fold folder State.Empty
        let back = line |> Seq.rev |> Seq.fold folder State.Empty

        front.visible 
        |> Seq.append back.visible
        |> Set.ofSeq

let lineToTree line column height =
    { line = line; column = column; height = $"{height}" |> int }

let forest = 
    System.IO.File.ReadLines(ResolutionFolder + "/input8.txt")
    |> Seq.mapi (fun line trees -> trees |> Seq.mapi (lineToTree line))

let lines = 
    forest 
    |> Seq.map Visible.find 
    |> Seq.concat

let columns = 
    forest 
    |> Seq.transpose 
    |> Seq.map Visible.find 
    |> Seq.concat

columns
|> Seq.append lines
|> Set.ofSeq
|> Seq.length

21

In [97]:
module Score =
    
    let backTrace height (trees: Tree seq) =
        trees 
        |> Seq.tryFindIndex (fun it -> it.height >= height)
        |> Option.map ((+) 1)
        |> Option.defaultValue (trees |> Seq.length)

    
    let getScore (visited) (tree: Tree) =
        let result = 
            if Seq.isEmpty visited then
                0
            else
                backTrace (tree.height) visited
        
        (tree, result), visited |> Seq.append [tree]

    let calculate trees =
        let forward =
            trees
            |> Seq.mapFold getScore Seq.empty
            |> fst
        let backward =
            trees
            |> Seq.rev
            |> Seq.mapFold getScore Seq.empty
            |> fst
            |> Seq.rev
        (forward,backward) ||> Seq.map2 (fun (ft,fs) (bt, bs) -> (fs * bs), ft)

// forest
// |> Seq.map (Seq.mapFold Score.getScore Seq.empty >> fst >> Seq.map snd) |> display

// forest
// |> Seq.map (Seq.rev >> Seq.mapFold Score.getScore Seq.empty >> fst >> Seq.map snd >> Seq.rev) |> display

let horizontal =
    forest
    |> Seq.map (Score.calculate)
    |> Seq.concat
    |> Seq.sortBy snd
    |> Seq.map fst

let vertical =
    forest
    |> Seq.transpose
    |> Seq.map (Score.calculate)
    |> Seq.concat
    |> Seq.sortBy snd
    |> Seq.map fst

(horizontal, vertical) ||> Seq.map2 (fun x y -> x * y) |> Seq.max

8